# $\tau$ VS $\eta$ (standard deviation of the Gaussian kernel versus normalized cut-off frequency)


Replicar esto, pero usando w1, h1 = signal.freqz(kernel[0], fs=10000)


In [ ]:
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from scipy import signal
from denoising.utils import get_gaussian_kernel

# Define parameters
beta = 0.1
sigma_values = np.linspace(1, 10, 100)  # Range of sigma values to test
N = 256  # Signal length
normalized_cutoff_freqs = []

for sigma in sigma_values:
    #print("sigma =", sigma)
    # Generate Gaussian kernel
    kernel = get_gaussian_kernel(sigma)
    #print(kernel.shape)
    #plt.plot(kernel)
    #plt.show()

    # Compute its frequency response
    freqs, response = signal.freqz(kernel, fs=10000)
    response = np.abs(response)
    #plt.plot(freqs, response)
    #plt.show()

    # Find the effective cut-off frequency (where magnitude falls below ~0.707)
    #cutoff_idx = np.where(response < 1 / np.sqrt(2))[0][0]
    #map = np.asarray(response < 1 / np.sqrt(2)).nonzero()
    #print(np.min(response))
    map = np.asarray(response < beta).nonzero()
    #print(map)
    try:
        cutoff_idx = map[0][0]
    except IndexError:
        cutoff_idx = 511
    #cutoff_idx = np.where(response < response[0] / np.sqrt(2))[0][0]
    #print(response[0] / 10)
    
    #cutoff_idx = np.where(response < response[0] / 10)[0][0]
    #print("freq_response[0]=", freq_response[0])
    #cutoff_idx = np.where(freq_response < 1.4)[0][0]
    #print("cutoff_idx=", cutoff_idx)
    cutoff_freq = freqs[cutoff_idx]/10000
    normalized_cutoff_freqs.append(cutoff_freq)

plt.plot(freqs, response)
plt.show()

# Fit an empirical formula: f_c^* ≈ C / sigma
coeff = np.polyfit(1 / sigma_values, normalized_cutoff_freqs, 1)

# Display empirical relationship
plt.figure(figsize=(6, 4))
#plt.plot(sigma_values, normalized_cutoff_freqs, 'bo-', label='Measured Cut-off')
plt.plot(normalized_cutoff_freqs, sigma_values, 'bo-', label='Measured Cut-off')
#plt.plot(sigma_values, coeff[0] * (1 / sigma_values) + coeff[1], 'r--', label=f'Fit: {coeff[0]:.3f}/sigma + {coeff[1]:.3f}')
plt.plot(coeff[0] * (1 / sigma_values) + coeff[1], sigma_values, 'r--', label=rf'Fit: {coeff[0]:.3f}/$\eta$ + {coeff[1]:.3f}')
plt.ylabel(r"$\tau$")
plt.xlabel(r"$\eta$")
plt.legend()
plt.title(rf"Empirical Relationship Between $\tau$ and $\eta$ for $beta={beta}$")
plt.grid()
plt.show()

print(f"Empirical formula: f_c^* ≈ {coeff[0]:.3f} / sigma + {coeff[1]:.3f}")


In [ ]:
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from denoising.utils import get_gaussian_kernel

# Define parameters
sigma_values = np.linspace(0.5, 10, 100)  # Range of sigma values to test
N = 256  # Signal length
normalized_cutoff_freqs = []

for sigma in sigma_values:
    # Generate Gaussian kernel
    kernel = get_gaussian_kernel(sigma)

    # Compute its frequency response
    freq_response = np.abs(np.fft.fft(kernel, N))  # Compute FFT
    freq_response = freq_response[:N//2]  # Take only the first half (positive frequencies)
    freq_axis = np.linspace(0, 0.5, N//2)  # Normalized frequency axis (Nyquist = 0.5)

    # Find the effective cut-off frequency (where magnitude falls below ~0.707)
    cutoff_idx = np.where(freq_response < freq_response[0] / np.sqrt(2))[0][0]
    #print("freq_response[0]=", freq_response[0])
    #cutoff_idx = np.where(freq_response < 1.4)[0][0]
    #print("cutoff_idx=", cutoff_idx)
    cutoff_freq = freq_axis[cutoff_idx]
    normalized_cutoff_freqs.append(cutoff_freq)

# Fit an empirical formula: f_c^* ≈ C / sigma
coeff = np.polyfit(1 / sigma_values, normalized_cutoff_freqs, 1)

# Display empirical relationship
plt.figure(figsize=(6, 4))
plt.plot(sigma_values, normalized_cutoff_freqs, 'bo-', label='Measured Cut-off')
plt.plot(sigma_values, coeff[0] * (1 / sigma_values) + coeff[1], 'r--', label=f'Fit: {coeff[0]:.3f}/sigma + {coeff[1]:.3f}')
plt.xlabel("Sigma")
plt.ylabel(r"Effective Normalized Cut-off Frequency ($f_c^*$)")
plt.legend()
plt.title("Empirical Relationship Between Sigma and Cut-off Frequency")
plt.grid()
plt.show()

print(f"Empirical formula: f_c^* ≈ {coeff[0]:.3f} / sigma + {coeff[1]:.3f}")


In [ ]:
def normalized_cutoff_to_tau(cutoff_freq):
    tau = 0.141 / cutoff_freq
    return tau

In [ ]:
tau = normalized_cutoff_to_tau(0.25)
tau

In [ ]:
import math
math.sqrt(2)

In [ ]:
np.sqrt(2)

In [ ]:
math.sqrt(-2*math.log(0.956))

In [ ]:
math.log(math.exp(10))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq

# Given parameters
tau = 5
fs = 100  # Sampling frequency
N = 256  # Length of the filter

# Generate Gaussian filter in discrete-time
n = np.arange(-N//2, N//2)  # Centered indices
h = np.exp(-n**2 / (2 * tau**2))  # Gaussian function

# Compute frequency response
H = np.abs(fft(h))  # Magnitude spectrum
freqs = fftfreq(N, d=1/fs)  # Frequency axis

# Compute cutoff frequency
eta = np.sqrt(-2 * np.log(0.1)) / tau
#eta_discrete = (eta / (2 * np.pi * fs)) * np.pi  # Discrete cutoff frequency
eta_discrete = (eta / (2 * fs))  # Discrete cutoff frequency

# Plot time-domain Gaussian filter
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(n, h, label="Gaussian filter")
plt.xlabel("Sample index (n)")
plt.ylabel("Amplitude")
plt.title("Discrete Gaussian Filter (Time Domain)")
plt.legend()
plt.grid()

# Plot frequency response
plt.subplot(1, 2, 2)
plt.plot(freqs[:N//2], H[:N//2], label="Magnitude Response")
plt.axvline(eta / (2 * np.pi), color='r', linestyle='--', label="Cutoff Frequency")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title("Frequency Response of Gaussian Filter")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()
